In [1]:
import os
import cv2
import json
import numpy as np 
import glob

In [2]:
curr = os.getcwd()
for i in range(1,101):
    img_name = "img" + str(i)
    source_folder = os.path.join(curr, "dataset-mask/" + img_name)    os.chdir(source_folder)
    
    for file in glob.glob("*_json.json"):
        os.rename(file, img_name + '_json.json')
    for file in glob.glob("*.csv"):
        os.rename(file, img_name + '_csv.csv')
    for file in glob.glob("*_coco.json"):
        os.rename(file, img_name + '_coco.json')       


C:\Users\hamsi\Desktop\Programming\AKCSE-Medical-Image-Analysis\dataset-mask/img1
C:\Users\hamsi\Desktop\Programming\AKCSE-Medical-Image-Analysis\dataset-mask/img2
C:\Users\hamsi\Desktop\Programming\AKCSE-Medical-Image-Analysis\dataset-mask/img3
C:\Users\hamsi\Desktop\Programming\AKCSE-Medical-Image-Analysis\dataset-mask/img4
C:\Users\hamsi\Desktop\Programming\AKCSE-Medical-Image-Analysis\dataset-mask/img5
C:\Users\hamsi\Desktop\Programming\AKCSE-Medical-Image-Analysis\dataset-mask/img6
C:\Users\hamsi\Desktop\Programming\AKCSE-Medical-Image-Analysis\dataset-mask/img7
C:\Users\hamsi\Desktop\Programming\AKCSE-Medical-Image-Analysis\dataset-mask/img8
C:\Users\hamsi\Desktop\Programming\AKCSE-Medical-Image-Analysis\dataset-mask/img9
C:\Users\hamsi\Desktop\Programming\AKCSE-Medical-Image-Analysis\dataset-mask/img10
C:\Users\hamsi\Desktop\Programming\AKCSE-Medical-Image-Analysis\dataset-mask/img11
C:\Users\hamsi\Desktop\Programming\AKCSE-Medical-Image-Analysis\dataset-mask/img12
C:\Users\hams

In [2]:
# Extract X and Y coordinates if available and update dictionary
def add_to_dict(data, itr, key, count):
    try:
        x_points = data[itr]["regions"][count]["shape_attributes"]["all_points_x"]
        y_points = data[itr]["regions"][count]["shape_attributes"]["all_points_y"]
    except:
        print("No BB. Skipping", key)
        return
    
    all_points = []
    for i, x in enumerate(x_points):
        all_points.append([x, y_points[i]])
    
    file_bbs[key] = all_points
    
file_bbs = {}                                      
MASK_WIDTH = 512
MASK_HEIGHT = 512

for i in range(1,101):
    img_name = "img" + str(i)
    source_folder = os.path.join(os.getcwd(), "dataset-mask/" + img_name)

    json_path = "dataset-mask/"+img_name+"/"+img_name+"_json.json"                    
    count = 0                                          

    # Read JSON file
    with open(json_path) as f:
      data = json.load(f)

    for itr in data:
        file_name_json = data[itr]["filename"]
        sub_count = 0               # Contains count of masks for a single ground truth image

        if len(data[itr]["regions"]) > 1:
            for _ in range(len(data[itr]["regions"])):
                key = file_name_json[:-4] + "*" + str(sub_count+1)
                add_to_dict(data, itr, key, sub_count)
                sub_count += 1
        else:
            add_to_dict(data, itr, file_name_json[:-4], 0)

    #print("\nDict size: ", len(file_bbs))

    file_name = img_name+".png"
    to_save_folder = os.path.join(source_folder, file_name[:-4])
    image_folder = os.path.join(to_save_folder, "images")
    mask_folder = os.path.join(to_save_folder, "masks")
    curr_img = os.path.join(source_folder, file_name)

    # make folders and copy image to new location
    os.mkdir(to_save_folder)
    os.mkdir(image_folder)
    os.mkdir(mask_folder)
    os.rename(curr_img, os.path.join(image_folder, file_name))

    # For each entry in dictionary, generate mask and save in correponding 
    # folder
    for itr in file_bbs:
        num_masks = itr.split("*")
        to_save_folder = os.path.join(source_folder, num_masks[0])
        mask_folder = os.path.join(to_save_folder, "masks")
        mask = np.zeros((MASK_WIDTH, MASK_HEIGHT))
        try:
            arr = np.array(file_bbs[itr])
        except:
            print("Not found:", itr)
            continue
        count += 1
        cv2.fillPoly(mask, [arr], color=(255))

        if len(num_masks) > 1:
            cv2.imwrite(os.path.join(mask_folder, itr.replace("*", "_") + ".png") , mask)    
        else:
            cv2.imwrite(os.path.join(mask_folder, itr + ".png") , mask)

print("Images saved:", count)

Images saved: 101


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\hamsi\\Desktop\\Programming\\AKCSE-Medical-Image-Analysis\\dataset-mask\\img1\\img1\\masks\\dataset-mask/img1/img1/masks'